In [141]:
import pandas as pd
import os
import json

from organizeEventData import *
from extract_game_info import *


In [142]:
country = 'france'

In [143]:
matchesFile = open(f"data/matches/matches_{country}.json")
matches = json.load(matchesFile)
matchesMap = {}
for match in matches:
    matchesMap[match["wyId"]] = match
matchesFile.close()

teamsFile = open("data/teams.json")
teams = json.load(teamsFile)
teamsFile.close()

playersFile = open("data/players.json")
players = json.load(playersFile)
playerMap = {}
for player in players:
    playerMap[player["wyId"]] =player
playersFile.close()

eventsFile = open(f"data/events/events_{country}.json")
events = json.load(eventsFile)
eventsFile.close()

In [144]:
df_matches = pd.read_json(f"/Users/tw/Desktop/Deep_Learning/Project/data/matches/matches_{country}.json")[['wyId', 'label', 'teamsData', 'winner']]

In [145]:
def clean_team_names(team):
    return team_dict[team]

def extract_label_information(label):
    teams, score = label.split(',')
    home_team, away_team = [x for x in teams.split(' - ')]
    home_goals, away_goals = [int(x.replace(' ', '')) for x in score.split(' - ')]

    return home_team, away_team, home_goals, away_goals

def get_team_ids(teamsData):
    return list(teamsData.keys())

def extract_game_information(teamsData, teams):
    teamIds = list(teamsData.keys())
    
    for t_id in teamIds:
        if teamsData[t_id]['side'] == 'home':
            team1_id = t_id
            team1_name = teamNameFromId(int(t_id), teams)
            team1_goals = teamsData[t_id]['score']
        elif teamsData[t_id]['side'] == 'away': 
            team2_id = t_id
            team2_name = teamNameFromId(int(t_id), teams)
            team2_goals = teamsData[t_id]['score']
        else:
            print('ERROR: team ids')
    return team1_id, team1_name, team2_id, team2_name, team1_goals, team2_goals

df_matches['team1_id'], df_matches['team1_name'], df_matches['team2_id'], df_matches['team2_name'], df_matches['team1_goals'], df_matches['team2_goals'] = zip(*df_matches.apply(lambda x: extract_game_information(x['teamsData'], teams), axis='columns'))
# df_matches['team1_id'], df_matches['team2_id'] = zip(*df_matches.apply(lambda x: get_team_ids(x['teamsData']), axis='columns'))

In [146]:
df_matches['team1_win'] = df_matches.apply(lambda x: int(x['winner']) == int(x['team1_id']), axis='columns')
df_matches['team2_win'] = df_matches.apply(lambda x: int(x['winner']) == int(x['team2_id']), axis='columns')

In [147]:
df_matches = df_matches.drop(columns=['label', 'teamsData', 'winner'])
df_matches = df_matches.rename(columns={"wyId": "game_id"})

In [148]:
df_matches.sort_values(by='game_id')

,game_id,team1_id,team1_name,team2_id,team2_name,team1_goals,team2_goals,team1_win,team2_win
379,2499719,1609,Arsenal,1631,Leicester City,4,3,True,False
372,2499720,1651,Brighton & Hove Albion,1625,Manchester City,0,2,False,True
373,2499721,1610,Chelsea,1646,Burnley,2,3,False,True
374,2499722,1628,Crystal Palace,1673,Huddersfield Town,0,3,False,True
375,2499723,1623,Everton,1639,Stoke City,1,0,True,False
...,...,...,...,...,...,...,...,...,...
5,2500094,1613,Newcastle United,1610,Chelsea,3,0,True,False
6,2500095,1619,Southampton,1625,Manchester City,0,1,False,True
7,2500096,10531,Swansea City,1639,Stoke City,1,2,False,True
8,2500097,1624,Tottenham Hotspur,1631,Leicester City,5,4,True,False


In [149]:
eventsPerMatch = groupEventsByMatch(events)

Done 10 matches
Done 20 matches
Done 30 matches
Done 40 matches
Done 50 matches
Done 60 matches
Done 70 matches
Done 80 matches
Done 90 matches
Done 100 matches
Done 110 matches
Done 120 matches
Done 130 matches
Done 140 matches
Done 150 matches
Done 160 matches
Done 170 matches
Done 180 matches
Done 190 matches
Done 200 matches
Done 210 matches
Done 220 matches
Done 230 matches
Done 240 matches
Done 250 matches
Done 260 matches
Done 270 matches
Done 280 matches
Done 290 matches
Done 300 matches
Done 310 matches
Done 320 matches
Done 330 matches
Done 340 matches
Done 350 matches
Done 360 matches
Done 370 matches
Done 380 matches


In [150]:
# list of dataframe rows (one per game)
game_events_rows = []

for row in df_matches.iterrows():
    match_id = row[1].game_id
    team1_id = row[1].team1_id
    team2_id = row[1].team2_id

    team1Events, team2Events = splitEventsByType(eventsPerMatch[match_id], [int(team1_id), int(team2_id)])

    team1NumDict = calcNumByEventType(team1Events)
    team2NumDict = calcNumByEventType(team2Events)

    team1_row = pd.json_normalize(team1NumDict, sep='_')
    team1_row = team1_row.add_prefix('team1_')
    team2_row = pd.json_normalize(team2NumDict, sep='_')
    team2_row = team2_row.add_prefix('team2_')

    game_row = pd.concat([team1_row, team2_row], axis=1)
    game_row['game_id'] = match_id
    game_events_rows.append(game_row)
    

In [151]:
game_events = pd.concat(game_events_rows, ignore_index=True)

In [152]:
df_matches = df_matches.merge(game_events, on='game_id')

In [153]:
df_matches

,game_id,team1_id,team1_name,team2_id,team2_name,team1_goals,team2_goals,team1_win,team2_win,team1_shot_5,...,team2_averagePassLength_50,team2_averagePassLength_55,team2_averagePassLength_60,team2_averagePassLength_65,team2_averagePassLength_70,team2_averagePassLength_75,team2_averagePassLength_80,team2_averagePassLength_85,team2_averagePassLength_90,team2_averagePassLength_90+
0,2500089,1646,Burnley,1659,AFC Bournemouth,1,2,False,True,0,...,24.414240,23.183810,19.273580,17.811794,25.365972,18.781827,25.493413,16.625699,21.450161,22.625026
1,2500090,1628,Crystal Palace,1627,West Bromwich Albion,2,0,True,False,0,...,24.378434,20.931002,21.014040,21.900460,23.410566,23.515655,25.370091,26.119361,23.121022,34.176015
2,2500091,1673,Huddersfield Town,1609,Arsenal,0,1,False,True,3,...,25.344614,19.488935,18.919828,20.788679,17.304366,21.366375,19.639208,25.043897,19.455295,19.451318
3,2500092,1612,Liverpool,1651,Brighton & Hove Albion,4,0,True,False,0,...,27.442228,21.212488,69.062880,30.888188,21.055327,22.694238,25.707501,12.786651,29.127068,30.056176
4,2500093,1611,Manchester United,1644,Watford,1,0,True,False,0,...,30.046974,23.763098,24.843168,19.398257,26.544659,22.430807,29.203424,28.610861,25.720594,26.092984
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2499723,1623,Everton,1639,Stoke City,1,0,True,False,0,...,26.238824,21.310861,21.455777,26.973639,24.845991,26.908126,24.732646,34.180548,18.572033,26.837983
376,2499726,1619,Southampton,10531,Swansea City,0,0,False,False,2,...,21.596198,31.038899,25.087964,26.090428,22.449752,26.264464,40.269619,29.826925,27.808304,16.061180
377,2499728,1627,West Bromwich Albion,1659,AFC Bournemouth,1,0,True,False,1,...,26.009628,23.888641,24.753764,24.452014,24.338005,22.507444,31.877006,20.914565,24.396551,26.912942
378,2499727,1644,Watford,1612,Liverpool,3,3,False,False,0,...,27.139081,26.296745,27.446299,16.318511,22.748270,19.125169,20.071687,29.044119,21.102642,34.201467


In [154]:
df_matches.sort_values(by=['game_id'])

,game_id,team1_id,team1_name,team2_id,team2_name,team1_goals,team2_goals,team1_win,team2_win,team1_shot_5,...,team2_averagePassLength_50,team2_averagePassLength_55,team2_averagePassLength_60,team2_averagePassLength_65,team2_averagePassLength_70,team2_averagePassLength_75,team2_averagePassLength_80,team2_averagePassLength_85,team2_averagePassLength_90,team2_averagePassLength_90+
379,2499719,1609,Arsenal,1631,Leicester City,4,3,True,False,1,...,24.701969,35.187854,14.081549,42.741662,29.810033,29.213555,19.680009,28.610095,20.106991,23.156835
372,2499720,1651,Brighton & Hove Albion,1625,Manchester City,0,2,False,True,0,...,21.472043,21.562735,21.805407,22.618485,23.957743,20.933805,25.235223,21.328526,23.955618,21.600367
373,2499721,1610,Chelsea,1646,Burnley,2,3,False,True,0,...,31.765202,28.588477,26.248809,32.671687,25.121740,55.578773,21.268862,28.949615,35.732518,23.851079
374,2499722,1628,Crystal Palace,1673,Huddersfield Town,0,3,False,True,0,...,27.240692,33.788129,24.279234,29.267694,34.746036,22.696938,21.574013,18.497958,17.007861,21.153259
375,2499723,1623,Everton,1639,Stoke City,1,0,True,False,0,...,26.238824,21.310861,21.455777,26.973639,24.845991,26.908126,24.732646,34.180548,18.572033,26.837983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,2500094,1613,Newcastle United,1610,Chelsea,3,0,True,False,0,...,27.273561,24.447097,23.783876,21.691170,23.144517,26.710452,21.968746,17.205163,22.735408,28.182454
6,2500095,1619,Southampton,1625,Manchester City,0,1,False,True,0,...,18.055655,25.284680,27.389164,19.855159,20.426304,23.643777,16.814633,23.482341,21.381297,26.596506
7,2500096,10531,Swansea City,1639,Stoke City,1,2,False,True,0,...,28.543939,28.487268,13.268032,24.297814,19.944191,19.672897,21.088770,27.156662,23.341779,0.000000
8,2500097,1624,Tottenham Hotspur,1631,Leicester City,5,4,True,False,0,...,28.235890,19.748974,15.715791,25.650377,21.602361,24.437060,24.595334,12.044429,25.376736,34.044914


In [155]:
# def getStartingLineupAverageHeight(match, playerMap):

#     teamIds = list(match["teamsData"].keys())
#     for t_id in teamIds:
#         if match["teamsData"][t_id]['side'] == 'home':
#             team1_id = t_id
#         else:
#             team2_id = t_id

#     team1Height = 0
#     team2Height = 0
#     for team in teamIds:
#         for player in match["teamsData"][team]["formation"]["lineup"]:
#             playerId = player["playerId"]
#             if (team == team1_id):
#                 team1Height+= playerMap[playerId]["height"]
#             elif (team == team2_id):
#                 team2Height+= playerMap[playerId]["height"]

#     return [team1_id, team1Height/11, team2_id, team2Height/11]

match_list = []
for matchId in df_matches['game_id']:
    match = [matchId] + getStartingLineupAverageHeight(matchesMap[matchId], playerMap)
    match_list.append(match)

In [156]:
df_height = pd.DataFrame(match_list, columns=['game_id', 'team1_id', 'team1_height', 'team2_id', 'team2_height'])

In [157]:
df_height

,game_id,team1_id,team1_height,team2_id,team2_height
0,2500089,1646,183.272727,1659,180.818182
1,2500090,1628,184.090909,1627,186.454545
2,2500091,1673,182.545455,1609,182.181818
3,2500092,1612,181.545455,1651,184.363636
4,2500093,1611,182.727273,1644,183.818182
...,...,...,...,...,...
375,2499723,1623,179.363636,1639,182.363636
376,2499726,1619,181.363636,10531,181.909091
377,2499728,1627,186.181818,1659,181.363636
378,2499727,1644,184.454545,1612,181.272727


In [158]:
df_matches = df_matches.merge(df_height, on=['game_id', 'team1_id', 'team2_id'])

In [159]:
df_matches.sort_values(by=['game_id'])

,game_id,team1_id,team1_name,team2_id,team2_name,team1_goals,team2_goals,team1_win,team2_win,team1_shot_5,...,team2_averagePassLength_60,team2_averagePassLength_65,team2_averagePassLength_70,team2_averagePassLength_75,team2_averagePassLength_80,team2_averagePassLength_85,team2_averagePassLength_90,team2_averagePassLength_90+,team1_height,team2_height
379,2499719,1609,Arsenal,1631,Leicester City,4,3,True,False,1,...,14.081549,42.741662,29.810033,29.213555,19.680009,28.610095,20.106991,23.156835,182.272727,181.454545
372,2499720,1651,Brighton & Hove Albion,1625,Manchester City,0,2,False,True,0,...,21.805407,22.618485,23.957743,20.933805,25.235223,21.328526,23.955618,21.600367,184.363636,181.727273
373,2499721,1610,Chelsea,1646,Burnley,2,3,False,True,0,...,26.248809,32.671687,25.121740,55.578773,21.268862,28.949615,35.732518,23.851079,183.181818,183.363636
374,2499722,1628,Crystal Palace,1673,Huddersfield Town,0,3,False,True,0,...,24.279234,29.267694,34.746036,22.696938,21.574013,18.497958,17.007861,21.153259,186.181818,184.181818
375,2499723,1623,Everton,1639,Stoke City,1,0,True,False,0,...,21.455777,26.973639,24.845991,26.908126,24.732646,34.180548,18.572033,26.837983,179.363636,182.363636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,2500094,1613,Newcastle United,1610,Chelsea,3,0,True,False,0,...,23.783876,21.691170,23.144517,26.710452,21.968746,17.205163,22.735408,28.182454,181.545455,183.363636
6,2500095,1619,Southampton,1625,Manchester City,0,1,False,True,0,...,27.389164,19.855159,20.426304,23.643777,16.814633,23.482341,21.381297,26.596506,183.636364,180.727273
7,2500096,10531,Swansea City,1639,Stoke City,1,2,False,True,0,...,13.268032,24.297814,19.944191,19.672897,21.088770,27.156662,23.341779,0.000000,180.272727,184.363636
8,2500097,1624,Tottenham Hotspur,1631,Leicester City,5,4,True,False,0,...,15.715791,25.650377,21.602361,24.437060,24.595334,12.044429,25.376736,34.044914,182.090909,183.363636


In [160]:
match_list = []
for matchId in df_matches['game_id']:
    match = [matchId] + getStartingLineupAverageAge(matchesMap[matchId], playerMap)
    match_list.append(match)

In [161]:
df_age = pd.DataFrame(match_list, columns=['game_id', 'team1_id', 'team1_age', 'team2_id', 'team2_age'])

In [162]:
df_matches = df_matches.merge(df_age, on=['game_id', 'team1_id', 'team2_id'])

In [163]:
df_matches.sort_values(by=['game_id'])

,game_id,team1_id,team1_name,team2_id,team2_name,team1_goals,team2_goals,team1_win,team2_win,team1_shot_5,...,team2_averagePassLength_70,team2_averagePassLength_75,team2_averagePassLength_80,team2_averagePassLength_85,team2_averagePassLength_90,team2_averagePassLength_90+,team1_height,team2_height,team1_age,team2_age
379,2499719,1609,Arsenal,1631,Leicester City,4,3,True,False,1,...,29.810033,29.213555,19.680009,28.610095,20.106991,23.156835,182.272727,181.454545,25.909091,28.000000
372,2499720,1651,Brighton & Hove Albion,1625,Manchester City,0,2,False,True,0,...,23.957743,20.933805,25.235223,21.328526,23.955618,21.600367,184.363636,181.727273,26.545455,27.000000
373,2499721,1610,Chelsea,1646,Burnley,2,3,False,True,0,...,25.121740,55.578773,21.268862,28.949615,35.732518,23.851079,183.181818,183.363636,26.454545,27.363636
374,2499722,1628,Crystal Palace,1673,Huddersfield Town,0,3,False,True,0,...,34.746036,22.696938,21.574013,18.497958,17.007861,21.153259,186.181818,184.181818,25.454545,24.909091
375,2499723,1623,Everton,1639,Stoke City,1,0,True,False,0,...,24.845991,26.908126,24.732646,34.180548,18.572033,26.837983,179.363636,182.363636,26.909091,27.272727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,2500094,1613,Newcastle United,1610,Chelsea,3,0,True,False,0,...,23.144517,26.710452,21.968746,17.205163,22.735408,28.182454,181.545455,183.363636,26.090909,26.363636
6,2500095,1619,Southampton,1625,Manchester City,0,1,False,True,0,...,20.426304,23.643777,16.814633,23.482341,21.381297,26.596506,183.636364,180.727273,26.181818,26.272727
7,2500096,10531,Swansea City,1639,Stoke City,1,2,False,True,0,...,19.944191,19.672897,21.088770,27.156662,23.341779,0.000000,180.272727,184.363636,28.818182,27.181818
8,2500097,1624,Tottenham Hotspur,1631,Leicester City,5,4,True,False,0,...,21.602361,24.437060,24.595334,12.044429,25.376736,34.044914,182.090909,183.363636,26.090909,28.636364


In [164]:
# # Reset DataFrame with columns in desired order
# cols = ['game_id','team1_name','team1_id','team2_name','team2_id']
# cols.extend(['team1_goals', 'team2_goals', 'team1_win', 'team2_win'])
# cols.extend(['team1_height', 'team2_height'])


# event_names = ['shot', 'corner', 'freeKickShots', 'redCard', 'yellowCard', 'offside']

# cols.extend([f'team1_{name}' for name in event_names])
# cols.extend([f'team2_{name}' for name in event_names])

# cols.extend(['shot', 'corner', 'freeKickShots', 'redCard', 'yellowCard', 'offside'])

# df_matches = df_matches[cols]

In [165]:
df_matches

,game_id,team1_id,team1_name,team2_id,team2_name,team1_goals,team2_goals,team1_win,team2_win,team1_shot_5,...,team2_averagePassLength_70,team2_averagePassLength_75,team2_averagePassLength_80,team2_averagePassLength_85,team2_averagePassLength_90,team2_averagePassLength_90+,team1_height,team2_height,team1_age,team2_age
0,2500089,1646,Burnley,1659,AFC Bournemouth,1,2,False,True,0,...,25.365972,18.781827,25.493413,16.625699,21.450161,22.625026,183.272727,180.818182,27.636364,25.727273
1,2500090,1628,Crystal Palace,1627,West Bromwich Albion,2,0,True,False,0,...,23.410566,23.515655,25.370091,26.119361,23.121022,34.176015,184.090909,186.454545,27.000000,29.272727
2,2500091,1673,Huddersfield Town,1609,Arsenal,0,1,False,True,3,...,17.304366,21.366375,19.639208,25.043897,19.455295,19.451318,182.545455,182.181818,26.454545,25.545455
3,2500092,1612,Liverpool,1651,Brighton & Hove Albion,4,0,True,False,0,...,21.055327,22.694238,25.707501,12.786651,29.127068,30.056176,181.545455,184.363636,24.727273,26.636364
4,2500093,1611,Manchester United,1644,Watford,1,0,True,False,0,...,26.544659,22.430807,29.203424,28.610861,25.720594,26.092984,182.727273,183.818182,27.909091,27.272727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2499723,1623,Everton,1639,Stoke City,1,0,True,False,0,...,24.845991,26.908126,24.732646,34.180548,18.572033,26.837983,179.363636,182.363636,26.909091,27.272727
376,2499726,1619,Southampton,10531,Swansea City,0,0,False,False,2,...,22.449752,26.264464,40.269619,29.826925,27.808304,16.061180,181.363636,181.909091,26.181818,27.454545
377,2499728,1627,West Bromwich Albion,1659,AFC Bournemouth,1,0,True,False,1,...,24.338005,22.507444,31.877006,20.914565,24.396551,26.912942,186.181818,181.363636,27.818182,27.181818
378,2499727,1644,Watford,1612,Liverpool,3,3,False,False,0,...,22.748270,19.125169,20.071687,29.044119,21.102642,34.201467,184.454545,181.272727,28.909091,25.181818


In [166]:
intervals = [
    ("5", (0,5)), 
    ("10", (5,10)), 
    ("15", (15, 20)), 
    ("20", (20, 25)),
    ("25", (25, 30)),
    ("30", (30, 35)),
    ("35", (35, 40)), 
    ("40", (40, 45)),
    ("45", (90, 95)),
    ("50", (95, 100)),
    ("55", (100, 105)),
    ("60", (105, 110)),
    ("65", (110, 115)), 
    ("70", (115, 120)),
    ("75", (120, 125)),
    ("80", (125, 130)),
    ("85", (130, 200))
]

dict_acc = {'game_id': [], 'team1_id': [], 'team2_id': []}
dict_invasion = {'game_id': [], 'team1_id': [], 'team2_id': []}

for idx, row in enumerate(df_matches.iterrows()):
    game_id = row[1].game_id
    team1_id = int(row[1].team1_id)
    team2_id = int(row[1].team2_id)
    
    a_match = []
    for ev in events:
        if ev['matchId'] == game_id:
            a_match.append(ev)

    list_invasion, list_acceleration = get_invasion_index(a_match,match_id,lst=True)

    dict_acc['game_id'].append(game_id)
    dict_invasion['game_id'].append(game_id)

    team_ids = [team1_id, team2_id]

    for team_id, team_label in zip(team_ids, ['team1', 'team2']):
        dict_acc[f'{team_label}_id'].append(team_id)
        dict_invasion[f'{team_label}_id'].append(team_id)
       
        df_acceleration = pd.DataFrame(list_acceleration[team_id], columns=['time','acceleration'])
        df_acceleration['time'] = [x/60. for x in df_acceleration.time]

        df_invasion = pd.DataFrame(list_invasion[team_id],columns=['time','invasion'])
        df_invasion['time'] = [x/60. for x in df_invasion.time]

        for intv_label, (start, end) in intervals:
            
            cond = ((df_acceleration['time'] > start) & (df_acceleration['time'] < end))
            within_interval = df_acceleration.loc[cond]

            key_1 = f'{team_label}_acc_{intv_label}'
            if key_1 in dict_acc:
                dict_acc[key_1].append(within_interval['acceleration'].mean())
            else:
                dict_acc[key_1] = [within_interval['acceleration'].mean()]

            
            cond = ((df_invasion['time'] > start) & (df_invasion['time'] < end))
            within_interval = df_invasion.loc[cond]

            key_2 = f'{team_label}_invasion_{intv_label}'

            if key_2 in dict_invasion:
                dict_invasion[key_2].append(within_interval['invasion'].mean())
            else:
                dict_invasion[key_2] = [within_interval['invasion'].mean()]

In [167]:
df_acc = pd.DataFrame(data=dict_acc)
df_acc = df_acc.astype({'team1_id': str, 'team2_id': str})
df_matches = df_matches.merge(df_acc, on=['game_id', 'team1_id', 'team2_id'])

In [168]:
df_inv = pd.DataFrame(data=dict_invasion)
df_inv = df_inv.astype({'team1_id': str, 'team2_id': str})
df_matches = df_matches.merge(df_inv, on=['game_id', 'team1_id', 'team2_id'])

In [169]:
df_matches

,game_id,team1_id,team1_name,team2_id,team2_name,team1_goals,team2_goals,team1_win,team2_win,team1_shot_5,...,team2_invasion_40,team2_invasion_45,team2_invasion_50,team2_invasion_55,team2_invasion_60,team2_invasion_65,team2_invasion_70,team2_invasion_75,team2_invasion_80,team2_invasion_85
0,2500089,1646,Burnley,1659,AFC Bournemouth,1,2,False,True,0,...,0.162667,0.490588,0.218333,0.283684,0.252143,0.340000,0.334118,0.308889,0.136429,0.138182
1,2500090,1628,Crystal Palace,1627,West Bromwich Albion,2,0,True,False,0,...,0.153000,0.241333,0.180556,0.311333,0.133500,0.318462,0.335333,0.465000,0.256500,0.213226
2,2500091,1673,Huddersfield Town,1609,Arsenal,0,1,False,True,3,...,0.087895,0.309444,0.200370,0.379333,0.164286,0.184706,0.256842,0.268571,0.273929,0.170851
3,2500092,1612,Liverpool,1651,Brighton & Hove Albion,4,0,True,False,0,...,0.330800,0.132500,0.284737,0.469091,0.197692,0.170000,0.004286,0.376522,0.105000,0.172778
4,2500093,1611,Manchester United,1644,Watford,1,0,True,False,0,...,0.482500,0.458000,0.291667,0.279167,0.165789,0.228824,0.210588,0.403529,0.376800,0.216389
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
375,2499723,1623,Everton,1639,Stoke City,1,0,True,False,0,...,0.422857,0.276667,0.129091,0.243667,0.118261,0.162632,0.328889,0.395263,0.433478,0.335926
376,2499726,1619,Southampton,10531,Swansea City,0,0,False,False,2,...,0.228421,0.485625,0.248571,0.147500,0.069412,0.295000,0.106000,0.224286,0.219524,0.247917
377,2499728,1627,West Bromwich Albion,1659,AFC Bournemouth,1,0,True,False,1,...,0.298571,0.274118,0.265200,0.470000,0.535556,0.396875,0.272632,0.301053,0.254000,0.339250
378,2499727,1644,Watford,1612,Liverpool,3,3,False,False,0,...,0.207143,0.190000,0.235000,0.397692,0.275714,0.267586,0.260588,0.207097,0.153000,0.231791


In [170]:
df_matches.to_csv(f'data/final_dfs/{country}_adv.csv', index=False)